In [18]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.outliers_influence import reset_ramsey
# import seaborn as sns
from scipy import stats
import re
# import scipy.stats as stats

In [19]:
source_file = 'Labour.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709
...,...,...,...,...
564,2.625403,20,1.424376,33.477545
565,1.276386,61,2.109048,26.300732
566,1.953869,117,6.241870,41.153979
567,1.318527,46,7.902237,66.720139


### 2 Сравнение моделей
#### (Вариант 2)

#### 2.2 output equations

Для набора данных `Labour` рассмотрим несколько моделей регрессии.



In [20]:
mod = []

In [21]:
formula = 'np.log(output) ~ np.log(capital) + np.log(labour) + np.log(wage)'
mod.append(smf.ols(formula = formula, data = df).fit())
formula2 = formula + ' + I(np.log(capital)**2)+I(np.log(labour)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + I(np.log(wage)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' + I(np.log(capital)**2)+I(np.log(labour)**2)+I(np.log(wage)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' - np.log(labour) + labour + I(labour**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())
formula2 = formula + ' - np.log(labour) + labour + I(labour**2) + I(np.log(wage)**2)'
mod.append(smf.ols(formula = formula2, data = df).fit())


In [22]:

print(
    summary_col(
        mod,
        float_format='%.3f',
        stars=True,
        model_names=['1', '2', '3', '4', '5', '6']
        # info_dict = info_dict
    )
)


                            1         2         3         4         5         6    
-----------------------------------------------------------------------------------
I(labour ** 2)                                                  -0.000*** -0.000***
                                                                (0.000)   (0.000)  
I(np.log(capital) ** 2)           0.008*              0.007                        
                                  (0.004)             (0.004)                      
I(np.log(labour) ** 2)            0.027***            0.028***                     
                                  (0.008)             (0.008)                      
I(np.log(wage) ** 2)                        0.141*    0.052               0.210*   
                                            (0.082)   (0.082)             (0.126)  
Intercept               -5.007*** -4.258*** -3.108*** -3.568*** -1.800*** 1.007    
                        (0.221)   (0.257)   (1.135)   (1.129)   (0.313)   (

In [23]:
df = pd.DataFrame()
for ind, model in enumerate(mod):
    df['mod' + str(ind+1)] = [
        model.rsquared_adj.round(3),
        model.aic.round(3),
        model.bic.round(3)
    ]
df

,mod1,mod2,mod3,mod4,mod5,mod6
0,0.888,0.895,0.888,0.895,0.742,0.743
1,567.246,532.626,566.314,534.227,1041.987,1041.197
2,584.622,558.689,588.034,564.635,1063.706,1067.260


In [24]:
df_res = pd.DataFrame(columns=['Метод', 'Регрессия'])
df_res.loc[0] = ['Adj.R2', df.loc[0].idxmax()]
df_res.loc[1] = ['AIC', df.loc[1].idxmin()]
df_res.loc[2] = ['BIC', df.loc[2].idxmin()]
df_res

,Метод,Регрессия
0,Adj.R2,mod2
1,AIC,mod2
2,BIC,mod2
